# Efficient solvers for semi-implicit hybridised DG methods in fluid dynamics
### _Jack Betteridge_★, Thomas Gibson† ,
### Ivan Graham★ , Lawrence Mitchell‡ , Eike Müller★
* ★Dept of Mathematical Sciences (Bath)
* †Imperial College London
* ‡Durham University

### Motivation
Atmospheric fluid dynamics separation of scales:
* Slow waves (≈ 30m/s): Coriolis force, advection, large scale weather patterns, quasi-geostrophic flow
* Fast waves (≈ 300m/s): Acoustic, dynamically "irrelevant" (carry little energy)

However...

* Fast waves are essential for driving large scale nonlinear dynamics
* Time step in explicit integrators for $\frac{\partial q}{\partial t} = \mathcal{N}(q(t))$ prohibitively small

⇒ semi-implicit methods (IMEX)
$$
\frac{\color{green}{q^{(t+\Delta t)}} - \color{blue}{q^{(t)}}}{\Delta t}
= \color{blue}{\underbrace{\mathcal{N}(q^{(t)}) - \mathcal{L}q^{(t)}}_{\text{explicit}}}\quad
+\color{green}{\underbrace{\mathcal{L}{q^{(t+\Delta t)}}}_{\text{implicit fast waves}}}
$$

### Discontinuous Galerkin Methods
**Key idea:** allow jumps in our solution

<center><img src="../images/DGschematic.svg" width="50%"></center>

and control jumps using a numerical flux.

## Information: **Advantages:**
* local data layout (no dofs on facets)
* arbitrary geometries
* inherently conservative
* sum-factorisation at higher order

**Linear system:**

$$\color{green}{(I − \Delta t L)q^{(t + \delta t)}} = \color{blue}{R(q^{(t)})}$$

**Problem:** Operator hard to precondition for DG problems

### Information: Our Method
* Hybridized DG method [Kang,Giraldo,Bui-Thanh(2017)]†
* With non-nested geometric multigrid on facet system [Gopalakrishnan,Tan(2009)]

†use expensive direct solver for facet system

### Non-linear Shallow Water Equations
<center><img src="../images/depth.png" width="50%"></center>
$$\begin{align*}
    \partial_t \color{green}{{u}}
    +\nabla\cdot \left(\frac{\color{green}{{u}}\otimes\color{green}{{u}}}{\color{green}{\phi}+\phi_B} 
    +\frac{(\color{green}{\phi}+\phi_B)^2}{2}I_2\right)
    &= \text{RHS}\\
    \partial_t\color{green}{\phi} + \hspace{4.7em}
    \nabla\cdot \color{green}{{u}} \hspace{5em}
    &= 0
\end{align*}$$

<center><video style="width:100%" autoplay=True loop=True controls class="w3-margin-bottom w3-center" src="../images/videos/bath_comp.mp4" /></center>

### DG Discretisation
<center><img src="../images/DGdiscretisation.svg" width="300px" class="w3-center"></center>
$$
\begin{equation*}
    \begin{pmatrix}
    A_{11} & A_{12}\\A_{21} & A_{22}
    \end{pmatrix}
    \begin{pmatrix}
    \color{green}{{U}^{(t+\Delta t)}}\\
    \color{green}{{\Phi}^{(t+\Delta t)}}
    \end{pmatrix} =
    \begin{pmatrix}
    \vec{R}_u\\
    \vec{R}_\phi
    \end{pmatrix}
\end{equation*}
$$

$A_{ij}$ **not** block-diagonal

### Hybridised DG Discretisation
<center><img src="../images/hybridizedDGdiscretisation.svg" width="300px"></center>
$$
\begin{pmatrix}
  \begin{array}{|@{\:}cc@{\:}|}
    \hline
    \hat{A}_{11} & \hat{A}_{12}\\
    \hat{A}_{21} & \hat{A}_{22} \\
    \hline
    \end{array}
  \begin{array}{c}\hat{A}_{13} \\ \hat{A}_{23}\end{array}\\
  \begin{array}{ccc}\hat{A}_{31} & \hat{A}_{32}\end{array}\quad \hat{A}_{33}
\end{pmatrix}
\begin{pmatrix}
\color{green}{{U}^{(t+\Delta t)}}\\
\color{green}{{\Phi}^{(t+\Delta t)}}\\
\color{red}{{\Lambda}}
\end{pmatrix}
= \begin{pmatrix}
\vec{R}_u\\
\vec{R}_\phi\\
\vec{R}_\lambda
\end{pmatrix}
$$

$A_{11},A_{12},A_{21},A_{22}$ **are** block-diagonal, $\color{red}{{\Lambda}}$ has fewer dofs than $\color{green}{{U}}$ or $\color{green}{{\Phi}}$ 

<center><img src="../images/sparsity.png" width="100%"></center>

### Non-nested Multigrid

<center><img src="../images/multigrid.svg" width="70%"></center>
Rediscretise problem on $P_1$ space

## Results

## Information : Software on Bath supercomputer *Balena:*
* _Firedrake_ finite element framework
* _SLATE_ language within Firedrake for hybridisation
* _PETSc_ multigrid solvers

**Stationary vortex analytic solution:**
<center><img src="../images/stationary_vortex.png" width="80%"></center>

**Single time step runtime per dof against unknowns:**
<center><img src="../images/NL_NOEXPruntime-ndof-ntime_ndof.png" width="800px"></center>
Non-linear stationary vortex, Lax-Friedrichs, Bathymetry, Theta Method

**Relative speedup of Hybridised AMG compared to DG
blocked preconditioner:**
<center><img src="../images/nonlinear_speedup_ndof.png" width="800px"></center>

## Information : Next steps:
* Spherical formulation
    * Tangential velocity, correct facet normal
* Matrix-free sum-factorised implementation
* Gravity wave/Navier-Stokes equations
    * Vertical anisotropy

<center><video style="width:50%" autoplay=True loop=True controls class="w3-margin-bottom w3-center" src="../images/videos/corner.mp4" /></center>

## Setup: Rendered using
<center><a href="https://github.com/JDBetteridge/nbfancy"><img src="../images/nbfancylogo.png" width="200px"></a></center>